## Script to create release locations of plastic from fishing activity
We use the Global Fishing Watch v2 Dataset found here: https://globalfishingwatch.org/data-download/datasets/public-fishing-effort

We first download the 'fleet-daily-csv-100-v2-YYYY.zip' files, and store them in the folder 'data/release/GlobalFishingWatch/'

We then run this script to create a .csv file with location, month, and summed fishing hours data.

In [1]:
import pandas as pd
import glob
import cartopy.io.shapereader as shpreader

In [2]:
input_data = '../../data/release/GlobalFishingWatch/'
output_data = '../../data/release/generated_files/'

In [3]:
files_fisheries = sorted(glob.glob(input_data+'fleet*/*') )

In [4]:
# New approach to storing the data - just store as lat/lon/month-year/sum(fishing_hours) - this is model independent!
data_fisheries = []
for file_ in files_fisheries:
    data_fisheries_day = pd.read_csv(file_)
    data_fisheries_day = data_fisheries_day[data_fisheries_day['fishing_hours'] > 0]
    data_fisheries.append(data_fisheries_day)

data_fisheries = pd.concat(data_fisheries, axis=0, ignore_index=True)

In [5]:
## Aggregate by location, flag, geartype, and date
agg_data_fisheries = data_fisheries.groupby(['cell_ll_lat', 'cell_ll_lon', 'flag', 'geartype', 'date'])['fishing_hours'].agg('sum').reset_index()

In [6]:
## Aggregate by location, flag, geartype, and month
agg_data_fisheries['date'] = pd.to_datetime(agg_data_fisheries['date'])
agg_data_fisheries['month'] = agg_data_fisheries['date'].values.astype('datetime64[M]')
agg_data_fisheries = agg_data_fisheries.groupby(['cell_ll_lat', 'cell_ll_lon', 'flag', 'geartype', 'month'])['fishing_hours'].agg('sum').reset_index()

In [7]:
# Store datafile
agg_data_fisheries.to_csv(output_data+'fleet_monthly_aggregated.csv')
## This chops it down from 70gb for one year to 2gb!

In [8]:
agg_data_fisheries

,cell_ll_lat,cell_ll_lon,flag,geartype,month,fishing_hours
0,-77.84,-177.56,NZL,set_longlines,2020-01-01,0.0336
1,-77.83,-177.66,NZL,set_longlines,2020-01-01,0.0830
2,-77.83,-177.65,NZL,set_longlines,2020-01-01,0.9102
3,-77.83,-177.57,NZL,set_longlines,2020-01-01,0.0836
4,-77.83,-177.56,NZL,set_longlines,2020-01-01,0.0652
...,...,...,...,...,...,...
54437360,81.69,35.01,EST,trawlers,2020-10-01,0.0333
54437361,81.69,35.03,EST,trawlers,2020-09-01,0.0175
54437362,81.69,35.03,EST,trawlers,2020-10-01,0.0188
54437363,81.70,34.70,EST,trawlers,2020-11-01,0.0499


In [9]:
## Match flag to country attributes
shpfilename = shpreader.natural_earth(resolution='50m',
                                      category='cultural',
                                      name='admin_0_countries')
reader = shpreader.Reader(shpfilename)
countries = reader.records()

countries_list = []
for country in countries:
    continent = country.attributes['CONTINENT']
    region_un = country.attributes['REGION_UN']
    subregion = country.attributes['SUBREGION']
    country_name = country.attributes['NAME_LONG']
    country_flag = country.attributes['ISO_A3']

    countries_list.append({'Continent': continent,
                            'Region': region_un,
                            'Subregion': subregion,
                            'Country': country_name,
                            'flag': country_flag})
countries_df = pd.DataFrame.from_records(countries_list)

In [10]:
# Combine datasets
agg_data_fisheries_info = pd.merge(agg_data_fisheries, countries_df, on="flag", how="left") #
agg_data_fisheries_info = agg_data_fisheries_info.rename(columns={"cell_ll_lat": "Latitude", "cell_ll_lon": "Longitude"})

In [11]:
# Store datafile
agg_data_fisheries_info.to_csv(output_data+'agg_data_fisheries_info.csv')
## This chops it down from 70gb for one year to 2gb!

In [12]:
agg_data_fisheries_info

,Latitude,Longitude,flag,geartype,month,fishing_hours,Continent,Region,Subregion,Country
0,-77.84,-177.56,NZL,set_longlines,2020-01-01,0.0336,Oceania,Oceania,Australia and New Zealand,New Zealand
1,-77.83,-177.66,NZL,set_longlines,2020-01-01,0.0830,Oceania,Oceania,Australia and New Zealand,New Zealand
2,-77.83,-177.65,NZL,set_longlines,2020-01-01,0.9102,Oceania,Oceania,Australia and New Zealand,New Zealand
3,-77.83,-177.57,NZL,set_longlines,2020-01-01,0.0836,Oceania,Oceania,Australia and New Zealand,New Zealand
4,-77.83,-177.56,NZL,set_longlines,2020-01-01,0.0652,Oceania,Oceania,Australia and New Zealand,New Zealand
...,...,...,...,...,...,...,...,...,...,...
54437360,81.69,35.01,EST,trawlers,2020-10-01,0.0333,Europe,Europe,Northern Europe,Estonia
54437361,81.69,35.03,EST,trawlers,2020-09-01,0.0175,Europe,Europe,Northern Europe,Estonia
54437362,81.69,35.03,EST,trawlers,2020-10-01,0.0188,Europe,Europe,Northern Europe,Estonia
54437363,81.70,34.70,EST,trawlers,2020-11-01,0.0499,Europe,Europe,Northern Europe,Estonia
